In [1]:
# ================================================================
#   NOTEBOOK XỬ LÝ DATASET CUSTOM2 → CHUẨN dataset_processed/
# ================================================================

import os
import cv2
import mediapipe as mp
import pandas as pd
from tqdm import tqdm

# ===============================
# CẤU HÌNH THƯ MỤC
# ===============================

DATA_ROOT = "CUSTOM2"                     # Folder gốc bạn đưa vào
OUTPUT_ROOT = "dataset_processed"         # Folder output chung
DOMAIN = "CUSTOM2"                         # Domain name
IMG_SIZE = 256
EXTRACT_EVERY = 10                         # lấy 1 frame mỗi 10 frames

# Tạo thư mục domain output đúng chuẩn:
OUTPUT_DOMAIN_DIR = os.path.join(OUTPUT_ROOT, DOMAIN)
LIVE_DIR  = os.path.join(OUTPUT_DOMAIN_DIR, "live")
SPOOF_DIR = os.path.join(OUTPUT_DOMAIN_DIR, "spoof")

os.makedirs(LIVE_DIR, exist_ok=True)
os.makedirs(SPOOF_DIR, exist_ok=True)

print("Output folders created:")
print(LIVE_DIR)
print(SPOOF_DIR)

# ===============================
# DETECTOR MẶT
# ===============================

mp_face = mp.solutions.face_detection

def detect_face(frame, detector):
    h, w = frame.shape[:2]
    results = detector.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    if not results.detections:
        return None

    det = results.detections[0]
    box = det.location_data.relative_bounding_box

    x1 = int(box.xmin * w)
    y1 = int(box.ymin * h)
    x2 = int((box.xmin + box.width) * w)
    y2 = int((box.ymin + box.height) * h)

    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = min(w, x2)
    y2 = min(h, y2)

    face = frame[y1:y2, x1:x2]
    if face.size == 0:
        return None

    face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
    return face


# ===============================
# XỬ LÝ 1 VIDEO
# ===============================

def process_video(video_path, is_live, csv_records):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("❌ Không mở được video:", video_path)
        return

    frame_id = 0
    saved = 0

    folder_out = LIVE_DIR if is_live else SPOOF_DIR

    with mp_face.FaceDetection(model_selection=0, min_detection_confidence=0.6) as detector:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_id % EXTRACT_EVERY == 0:
                face = detect_face(frame, detector)
                if face is not None:
                    base = os.path.splitext(os.path.basename(video_path))[0]
                    img_name = f"{base}_{frame_id}.jpg"
                    save_path = os.path.join(folder_out, img_name)

                    cv2.imwrite(save_path, face)

                    rel_path = f"{DOMAIN}/{('live' if is_live else 'spoof')}/{img_name}"
                    csv_records.append([rel_path, int(is_live), DOMAIN])

                    saved += 1

            frame_id += 1

    cap.release()
    print(f"✓ {os.path.basename(video_path)} → {saved} ảnh")


# ===============================
# XỬ LÝ 2 FOLDER train/test
# ===============================

def process_split(split_name, csv_records):
    split_dir = os.path.join(DATA_ROOT, split_name)

    real_dir = os.path.join(split_dir, "real_video")
    attack_dir = os.path.join(split_dir, "attack")

    print(f"\n========== PROCESSING {split_name.upper()} ==========")

    # REAL → live
    if os.path.exists(real_dir):
        reals = [f for f in os.listdir(real_dir) if f.endswith(".mp4")]
        for v in tqdm(reals, desc=f"{split_name} real"):
            process_video(os.path.join(real_dir, v), is_live=True, csv_records=csv_records)

    # ATTACK → spoof
    if os.path.exists(attack_dir):
        spoofs = [f for f in os.listdir(attack_dir) if f.endswith(".mp4")]
        for v in tqdm(spoofs, desc=f"{split_name} spoof"):
            process_video(os.path.join(attack_dir, v), is_live=False, csv_records=csv_records)


# ===============================
# CHẠY TOÀN BỘ
# ===============================

csv_records = []

process_split("train", csv_records)
process_split("test", csv_records)

# ===============================
# LƯU FILE CSV DOMAIN
# ===============================

csv_path = os.path.join(OUTPUT_ROOT, f"{DOMAIN}_list.csv")
df = pd.DataFrame(csv_records, columns=["path", "label", "domain"])
df.to_csv(csv_path, index=False)

print("\n====================================")
print("  ✓ HOÀN THÀNH")
print("  → CSV saved:", csv_path)
print("====================================")


Output folders created:
dataset_processed\CUSTOM2\live
dataset_processed\CUSTOM2\spoof

========== PROCESSING TRAIN ==========


train real:   2%|▏         | 1/55 [00:00<00:22,  2.36it/s]

✓ 0.mp4 → 41 ảnh


train real:   4%|▎         | 2/55 [00:01<00:56,  1.07s/it]

✓ 1.mp4 → 28 ảnh


...

✓ 77.mp4 → 44 ảnh


test spoof:  96%|█████████▌| 24/25 [01:02<00:02,  2.45s/it]

✓ 78.mp4 → 32 ảnh


test spoof: 100%|██████████| 25/25 [01:05<00:00,  2.62s/it]

✓ 79.mp4 → 45 ảnh

  ✓ HOÀN THÀNH
  → CSV saved: dataset_processed\CUSTOM2_list.csv
